In [1]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from lib.dataset import split_training_validation
import pandas as pd
from lib.trees import get_tree,parse_edge_list
import random
from collections import defaultdict
import networkx as nx
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier


2025-05-29 00:12:46.312547: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load data

In [2]:
sentences = pd.read_csv("../data/train.csv")
sentences["language"] = sentences["language"].astype("category")
sentences["edgelist"] = sentences["edgelist"].apply(parse_edge_list)
sentences["tree"] = sentences["edgelist"].apply(get_tree)
sentences.head()

,language,sentence,n,edgelist,root,tree
0,Japanese,2,23,"[(6, 4), (2, 6), (2, 23), (20, 2), (15, 20), (...",10,"(6, 4, 2, 23, 20, 15, 3, 5, 14, 8, 12, 9, 18, ..."
1,Japanese,5,18,"[(8, 9), (14, 8), (4, 14), (5, 4), (1, 2), (6,...",10,"(8, 9, 14, 4, 5, 1, 2, 6, 17, 12, 3, 7, 11, 16..."
2,Japanese,8,33,"[(2, 10), (2, 14), (4, 2), (16, 4), (6, 16), (...",3,"(2, 10, 14, 4, 16, 6, 12, 32, 26, 3, 29, 27, 2..."
3,Japanese,11,30,"[(30, 1), (14, 24), (21, 14), (3, 21), (7, 3),...",30,"(30, 1, 14, 24, 21, 3, 7, 12, 27, 16, 8, 5, 26..."
4,Japanese,12,19,"[(19, 13), (16, 19), (2, 16), (4, 10), (4, 15)...",11,"(19, 13, 16, 2, 4, 10, 15, 5, 14, 12, 3, 1, 8,..."


# Split data

In [3]:
random.seed(121)
training, validation = split_training_validation(sentences, 0.2)

print("Training set size:", len(training))
print("Validation set size:", len(validation))

Training set size: 8400
Validation set size: 2100


In [4]:
from typing import List


def get_longest_paths(graph) -> List[List]:
    """
    Find all longest simple path in an undirected graph.
    """
    longest_paths = []
    longest_path_length = 0
    for source in graph.nodes():
        for target in graph.nodes():
            if source == target:
                continue
            # Find all simple paths between source and target
            paths = nx.all_simple_paths(graph, source=source, target=target)
            for path in paths:
                if len(path) > longest_path_length:
                    longest_path_length = len(path)
                    longest_paths.clear()
                    longest_paths.append(path)
                elif len(path) == longest_path_length:
                    longest_paths.append(path)
    return longest_paths


def unwind_tree(row: pd.Series) -> pd.DataFrame:
    """
    Unwind a tree into a list of nodes, with their centrality scores.
    """
    tree: nx.Graph = row["tree"]
    root_node = row["root"]
    language = row["language"]

    # Centrality measures
    eccentricity = nx.eccentricity(tree)  # Maximum distance to any other node in the tree.
    closeness_centrality = nx.closeness_centrality(tree)  # Reciprocal avg distance to all other nodes in the tree.
    degree_centrality = nx.degree_centrality(tree)  # Fraction of nodes that a node is connected to.
    betweenness_centrality = nx.betweenness_centrality(tree)  # Fraction of shortest paths that pass through.
    harmonic_centrality = nx.harmonic_centrality(tree)  # Average distance to all other nodes in the tree.
    pagerank = nx.pagerank(tree)  # PageRank algorithm, which measures the importance of nodes.
    katz_centrality = nx.katz_centrality(tree)  # It's a generalization of eigenvector centrality.
    current_flow_closeness = nx.current_flow_closeness_centrality(tree)  # Closeness cent. based on resistance.
    current_flow_betweenness = nx.current_flow_betweenness_centrality(tree)  # Betweenness cent. based on resistance.
    load_centrality = nx.load_centrality(tree)  # Similar to betweenness centrality.
    percolation_centrality = nx.percolation_centrality(tree)  # Proportion of "percolation" paths through a node.
    second_order_centrality = nx.second_order_centrality(tree)  # Std of return times in perpetual random walk.
    laplacian_centrality = nx.laplacian_centrality(tree)  # Centrality based on the Laplacian matrix.

    # Tree properties
    tree_diameter = nx.diameter(tree)  # Length of the longest path in the tree.
    # Get the centroids of the tree
    centroids = nx.center(tree)
    # Get the leaves of the tree
    leaves = [node for node, degree in tree.degree() if degree == 1]
    # Get the longest path in the tree
    longest_path = get_longest_paths(tree)
    longest_path_nodes = set()
    for path in longest_path:
        longest_path_nodes.update(path)

    rows = []
    for node in tree:
        # Create a tree rooted at the current node
        tree_rooted_at_node: nx.DiGraph = nx.bfs_tree(tree, source=node)
        # We can do this because we know it will be a tree, so no cycles exist
        # and thus, bfs_edges will give us a valid tree structure.

        subtrees: list[nx.DiGraph] = []
        for child in tree.neighbors(node):
            subtrees.append(nx.bfs_tree(tree, source=child))

        subtree_leaves = []
        for subtree in subtrees:
            subtree_leaves.append([n for n in subtree if subtree.out_degree[n] == 0])

        subtree_depths = []
        for subtree in subtrees:
            subtree_depths.append(nx.dag_longest_path_length(subtree))

        rows.append(
            {
                "row_index": row.name,
                "node": node,
                "is_root": node == root_node,
                # Global properties
                "language": language,
                "tree_diameter": tree_diameter,
                "tree_size": tree.number_of_nodes(),
                "tree_edges": tree.number_of_edges(),
                "number_of_centroids": len(centroids),
                "average_degree": sum(dict(tree.degree()).values()) / tree.number_of_nodes(),
                "number_of_leaves": len(leaves),
                # Local Node Properties
                "degree": tree.degree[node],
                "is_leaf": tree.degree[node] == 1,  # Only a leaf can have degree 1
                "is_centroid": node in centroids,
                # Path properties
                "distance_to_closest_centroid": min(
                    nx.shortest_path_length(tree, source=c, target=node) for c in centroids
                ),
                "distance_to_farthest_centroid": max(
                    nx.shortest_path_length(tree, source=c, target=node) for c in centroids
                ),
                "distance_to_closest_leaf": min(nx.shortest_path_length(tree, source=l, target=node) for l in leaves),
                "distance_to_farthest_leaf": max(nx.shortest_path_length(tree, source=l, target=node) for l in leaves),
                "is_in_longest_path": node in longest_path_nodes,
                # Properties of tree rooted at this node
                "tree_depth_if_root": nx.dag_longest_path_length(tree_rooted_at_node),
                "min_subtree_size_if_root": min(subtree.number_of_nodes() for subtree in subtrees) if subtrees else 0,
                "max_subtree_size_if_root": max(subtree.number_of_nodes() for subtree in subtrees) if subtrees else 0,
                "average_subtree_size_if_root": (
                    sum(subtree.number_of_nodes() for subtree in subtrees) / len(subtrees) if subtrees else 0
                ),
                "min_subtree_leaf_count_if_root": (min(len(l) for l in subtree_leaves) if subtree_leaves else 0),
                "max_subtree_leaf_count_if_root": (max(len(l) for l in subtree_leaves) if subtree_leaves else 0),
                "average_subtree_leaf_count_if_root": (
                    sum(len(l) for l in subtree_leaves) / len(subtree_leaves) if subtree_leaves else 0
                ),
                "min_subtree_depth_if_root": (min(subtree_depths) if subtree_depths else 0),
                "max_subtree_depth_if_root": (max(subtree_depths) if subtree_depths else 0),
                "average_subtree_depth_if_root": (sum(subtree_depths) / len(subtree_depths) if subtree_depths else 0),
                "depth_difference_if_root": (max(subtree_depths) - min(subtree_depths)) if subtree_depths else 0,
                "number_of_subtrees_if_root": len(subtrees),
                # Centrality measures
                "eccentricity": eccentricity[node],
                "closeness_centrality": closeness_centrality[node],
                #"closeness_centrality_inverse": 1 / closeness_centrality[node] if closeness_centrality[node] > 0 else 0,
                "degree_centrality": degree_centrality[node],
                "harmonic_centrality": harmonic_centrality[node],
                "betweenness_centrality": betweenness_centrality[node],
                "pagerank": pagerank[node],
                "katz_centrality": katz_centrality[node],
                "current_flow_closeness": current_flow_closeness[node],
                "current_flow_betweenness": current_flow_betweenness[node],
                "load_centrality": load_centrality[node],
                "percolation_centrality": percolation_centrality[node],
                "second_order_centrality": second_order_centrality[node],
                "laplacian_centrality": laplacian_centrality[node],
            }
        )
    return pd.DataFrame(rows)

In [5]:
# validation_unwound = pd.concat(validation.apply(unwind_tree, axis=1).tolist(), ignore_index=True)
# validation_unwound["language"] = validation_unwound["language"].astype("category")
# validation_unwound.to_csv("../data/cache/validation_unwound.csv", index=False)
# validation_unwound.head()

validation_unwound = pd.read_csv("../data/cache/validation_unwound.csv")

In [6]:
# training_unwound = pd.concat(training.apply(unwind_tree, axis=1).tolist(), ignore_index=True)
# training_unwound["language"] = training_unwound["language"].astype("category")
# training_unwound.to_csv("../data/cache/training_unwound.csv", index=False)
# training_unwound.head()

training_unwound = pd.read_csv("../data/cache/training_unwound.csv")


In [7]:
# training_unwound.to_csv("../data/cache/biel/training_unwound.csv", index=False)
# validation_unwound.to_csv("../data/cache/biel/validation_unwound.csv", index=False)

In [10]:
X_train = training_unwound.drop(columns=["row_index","node","is_root"])
X_train = pd.get_dummies(X_train, columns=["language"], drop_first=False) 
y_train = training_unwound["is_root"]


X_val = validation_unwound.drop(columns=["row_index","node","is_root",])
X_val = pd.get_dummies(X_val, columns=["language"], drop_first=False)
y_val = validation_unwound["is_root"]

In [17]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

predictions = lda.predict(X_val)
print(f"Node-based accuracy: {accuracy_score(y_val, predictions):.2f}")


sentence_predictions = defaultdict(dict)
probs = lda.predict_proba(X_val)

sentence_real_root = {}
for (_, row), pred in zip(validation_unwound.iterrows(), probs):
    sentence_predictions[row["row_index"]][row["node"]] = pred[1]
    if row["is_root"]:
        sentence_real_root[row["row_index"]] = row["node"]

if not set(sentence_predictions.keys()) == set(sentence_real_root.keys()):
    raise ValueError("Mismatch between sentence predictions and real roots.")


def get_predicted_root(row: pd.Series) -> str:
    """
    Get the predicted root node for a sentence.
    """
    sentence_id = row.name
    probs = sentence_predictions[sentence_id]
    return max(probs.keys(), key=probs.get)


validation_prediction = pd.DataFrame.from_dict(sentence_real_root, orient="index", columns=["root"])
validation_prediction["predicted_root"] = validation_prediction.apply(get_predicted_root, axis=1)
print(
    f"Sentence-based accuracy: {accuracy_score(validation_prediction['root'], validation_prediction['predicted_root']):.2f}"
)

Node-based accuracy: 0.93
Sentence-based accuracy: 0.26


In [18]:
qda =  QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

predictions = qda.predict(X_val)
print(f"Node-based accuracy: {accuracy_score(y_val, predictions):.2f}")

sentence_predictions = defaultdict(dict)
probs = qda.predict_proba(X_val)

sentence_real_root = {}
for (_, row), pred in zip(validation_unwound.iterrows(), probs):
    sentence_predictions[row["row_index"]][row["node"]] = pred[1]
    if row["is_root"]:
        sentence_real_root[row["row_index"]] = row["node"]

if not set(sentence_predictions.keys()) == set(sentence_real_root.keys()):
    raise ValueError("Mismatch between sentence predictions and real roots.")


def get_predicted_root(row: pd.Series) -> str:
    """
    Get the predicted root node for a sentence.
    """
    sentence_id = row.name
    probs = sentence_predictions[sentence_id]
    return max(probs.keys(), key=probs.get)


validation_prediction = pd.DataFrame.from_dict(sentence_real_root, orient="index", columns=["root"])
validation_prediction["predicted_root"] = validation_prediction.apply(get_predicted_root, axis=1)
print(
    f"Sentence-based accuracy: {accuracy_score(validation_prediction['root'], validation_prediction['predicted_root']):.2f}"
)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Node-based accuracy: 0.81
Sentence-based accuracy: 0.19


In [19]:
nb = GaussianNB()
nb.fit(X_train, y_train)


predictions = nb.predict(X_val)
print(f"Node-based accuracy: {accuracy_score(y_val, predictions):.2f}")


sentence_predictions = defaultdict(dict)
probs = nb.predict_proba(X_val)

sentence_real_root = {}
for (_, row), pred in zip(validation_unwound.iterrows(), probs):
    sentence_predictions[row["row_index"]][row["node"]] = pred[1]
    if row["is_root"]:
        sentence_real_root[row["row_index"]] = row["node"]

if not set(sentence_predictions.keys()) == set(sentence_real_root.keys()):
    raise ValueError("Mismatch between sentence predictions and real roots.")


def get_predicted_root(row: pd.Series) -> str:
    """
    Get the predicted root node for a sentence.
    """
    sentence_id = row.name
    probs = sentence_predictions[sentence_id]
    return max(probs.keys(), key=probs.get)


validation_prediction = pd.DataFrame.from_dict(sentence_real_root, orient="index", columns=["root"])
validation_prediction["predicted_root"] = validation_prediction.apply(get_predicted_root, axis=1)
print(
    f"Sentence-based accuracy: {accuracy_score(validation_prediction['root'], validation_prediction['predicted_root']):.2f}"
)

Node-based accuracy: 0.78
Sentence-based accuracy: 0.27


In [20]:
lr = LogisticRegression(max_iter=5000)
lr.fit(X_train, y_train)


predictions = lr.predict(X_val)
print(f"Node-based accuracy: {accuracy_score(y_val, predictions):.2f}")


sentence_predictions = defaultdict(dict)
probs = lr.predict_proba(X_val)

sentence_real_root = {}
for (_, row), pred in zip(validation_unwound.iterrows(), probs):
    sentence_predictions[row["row_index"]][row["node"]] = pred[1]
    if row["is_root"]:
        sentence_real_root[row["row_index"]] = row["node"]

if not set(sentence_predictions.keys()) == set(sentence_real_root.keys()):
    raise ValueError("Mismatch between sentence predictions and real roots.")


def get_predicted_root(row: pd.Series) -> str:
    """
    Get the predicted root node for a sentence.
    """
    sentence_id = row.name
    probs = sentence_predictions[sentence_id]
    return max(probs.keys(), key=probs.get)


validation_prediction = pd.DataFrame.from_dict(sentence_real_root, orient="index", columns=["root"])
validation_prediction["predicted_root"] = validation_prediction.apply(get_predicted_root, axis=1)
print(
    f"Sentence-based accuracy: {accuracy_score(validation_prediction['root'], validation_prediction['predicted_root']):.2f}"
)

Node-based accuracy: 0.95
Sentence-based accuracy: 0.27


In [ ]:
svm = SVC()
svm.fit(X_train, y_train)

predictions = svm.predict(X_val)
print(f"Node-based accuracy: {accuracy_score(y_val, predictions):.2f}")


sentence_predictions = defaultdict(dict)
probs = svm.predict_proba(X_val)
sentence_real_root = {}

for (_, row), pred in zip(validation_unwound.iterrows(), probs):
    sentence_predictions[row["row_index"]][row["node"]] = pred[1]
    if row["is_root"]:
        sentence_real_root[row["row_index"]] = row["node"]
if not set(sentence_predictions.keys()) == set(sentence_real_root.keys()):
    raise ValueError("Mismatch between sentence predictions and real roots.")
def get_predicted_root(row: pd.Series) -> str:
    """
    Get the predicted root node for a sentence.
    """
    sentence_id = row.name
    probs = sentence_predictions[sentence_id]
    return max(probs.keys(), key=probs.get)
validation_prediction = pd.DataFrame.from_dict(sentence_real_root, orient="index", columns=["root"])
validation_prediction["predicted_root"] = validation_prediction.apply(get_predicted_root, axis=1)
print(
    f"Sentence-based accuracy: {accuracy_score(validation_prediction['root'], validation_prediction['predicted_root']):.2f}"
)

In [9]:
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train, y_train)


predictions = xgb.predict(X_val)
print(f"Node-based accuracy: {accuracy_score(y_val, predictions):.2f}")



sentence_predictions = defaultdict(dict)
probs = xgb.predict_proba(X_val)
sentence_real_root = {}

for (_, row), pred in zip(validation_unwound.iterrows(), probs):
    sentence_predictions[row["row_index"]][row["node"]] = pred[1]
    if row["is_root"]:
        sentence_real_root[row["row_index"]] = row["node"]
if not set(sentence_predictions.keys()) == set(sentence_real_root.keys()):
    raise ValueError("Mismatch between sentence predictions and real roots.")

def get_predicted_root(row: pd.Series) -> str:
    """
    Get the predicted root node for a sentence.
    """
    sentence_id = row.name
    probs = sentence_predictions[sentence_id]
    return max(probs.keys(), key=probs.get)
validation_prediction = pd.DataFrame.from_dict(sentence_real_root, orient="index", columns=["root"])
validation_prediction["predicted_root"] = validation_prediction.apply(get_predicted_root, axis=1)
print(
    f"Sentence-based accuracy: {accuracy_score(validation_prediction['root'], validation_prediction['predicted_root']):.2f}"
)

Node-based accuracy: 0.95
Sentence-based accuracy: 0.27


In [13]:
#tune xgboost
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
grid_search = GridSearchCV(XGBClassifier(eval_metric='logloss'), param_grid, cv=3, scoring='f1', verbose=1)
grid_search.fit(X_train, y_train)
print("Best parameters found: ", grid_search.best_params_)
best_xgb = grid_search.best_estimator_
predictions = best_xgb.predict(X_val)
print(f"Node-based accuracy after tuning: {accuracy_score(y_val, predictions):.2f}")
sentence_predictions = defaultdict(dict)
probs = best_xgb.predict_proba(X_val)
sentence_real_root = {}

for (_, row), pred in zip(validation_unwound.iterrows(), probs):
    sentence_predictions[row["row_index"]][row["node"]] = pred[1]
    if row["is_root"]:
        sentence_real_root[row["row_index"]] = row["node"]
if not set(sentence_predictions.keys()) == set(sentence_real_root.keys()):
    raise ValueError("Mismatch between sentence predictions and real roots.")

def get_predicted_root(row: pd.Series) -> str:
    """
    Get the predicted root node for a sentence.
    """
    sentence_id = row.name
    probs = sentence_predictions[sentence_id]
    return max(probs.keys(), key=probs.get)
validation_prediction = pd.DataFrame.from_dict(sentence_real_root, orient="index", columns=["root"])
validation_prediction["predicted_root"] = validation_prediction.apply(get_predicted_root, axis=1)
print(
    f"Sentence-based accuracy after tuning: {accuracy_score(validation_prediction['root'], validation_prediction['predicted_root']):.2f}"
)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters found:  {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}
Node-based accuracy after tuning: 0.95
Sentence-based accuracy after tuning: 0.26
